In [1]:
import pandas as pd              # v1.4.3
import numpy as np               # v1.20.1
import matplotlib.pyplot as plt  # v3.5.2
import folium                    # v0.12.1.post1

In [2]:
# Should really use glob to read this all in. Next time.....
# Can't set dtypes cause things I want to be 'int16' have NaNs. Prefer finer grained dropna later than on read
Parcels_09 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2009.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_10 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2010.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_11 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2011.csv', sep=',', index_col='OBJECTID', low_memory=False)

In [11]:
Parcels_12 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2012.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_13 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2013.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_14 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2014.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_15 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2015.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_16 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2016.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_17 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2017.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_18 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2018.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_19 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2019.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_20 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2020.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_21 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2021.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_22 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2022.csv', sep=',', index_col='OBJECTID', low_memory=False)

In [4]:
Parcels_09.info(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343897 entries, 1 to 343897
Data columns (total 33 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   DIGEST         343897 non-null  int16  
 1   SITUS          343897 non-null  object 
 2   TAXPIN         343897 non-null  object 
 3   PARID          343897 non-null  object 
 4   ATRPIN         343897 non-null  object 
 5   TAX_DISTR      343897 non-null  object 
 6   OWNER1         343897 non-null  object 
 7   OWNER2         343897 non-null  object 
 8   ADD2           343897 non-null  object 
 9   ADD3           343897 non-null  object 
 10  ADD4           343897 non-null  object 
 11  ADD5           343897 non-null  object 
 12  LUC            343897 non-null  object 
 13  NBHD           343897 non-null  object 
 14  PROP_CLASS     343897 non-null  object 
 15  CLASS          343897 non-null  object 
 16  TOT_APPR       343897 non-null  int64  
 17  TOT_ASSESS     343897 non-nul

In [7]:
# check ['TAXYEAR'] data
Parcels_09.TAXYEAR.head()

OBJECTID
1    2009
2    2009
3    2009
4    2009
5    2009
Name: TAXYEAR, dtype: int64

# The Plan  
- [x] Get property records  
- [x] Clean data  
    - [x] Fix ['Columns']   
        - [x] Unify column names  
    - [x] Concat years into single dataframe  
    - [x] Drop ['Columns'] with lots of NaNs  
        - NaN > 25k seems to be enough to get rid of empty columns but also keep semi-useful data (street suffixes)
            - Ended up using 710k since the entire data set has +4M rows. The important data has missing data < 1000.
    - [x] df.dropna(subset=['columns']) rows with lots of NaNs   
        - ['ParcelID', 'TaxDist', 'TotAppr', 'TotAssess', 'LivUnits']   
    - [x] Export AllYears.csv (don't use that name >< )
    - [x] Drop non-ATL properties  
        - [x] Export ATL-PropertyRecords.csv  

## Parcel data information  
The CAMA information included with the published dataset is selected based on its value to the typical consumer of the data and includes the parcel identification number, the property address, property owner, owner's mailing address, tax district, assessed and appraised value for land and improvements, the number of livable units, acreage, property class and land use class.

Fulton County GIS Collaboration Group [Centerline Address Landmark Data Model](http://share.myfultoncountyga.us/datashare/fultoncounty/Documents/CenterlineAddressLandmarkDataModelDocument.pdf)

In [12]:
# a list of parcel tax IDs in the City of Atlanta.
atlanta_tax_IDs = ['05', '05A', '05B', '05C', '05D', '05E', '05F', '05G', '05H', '05I', '05J',
                    '05K', '05L', '05P', '05Q', '05R', '05S', '05T', '05U', '05V', '05W', '05X',
                  '05Y', '05Z']  

# column names changed in 2011, renaming to make analysis easier. I wish I could just use pd.str functions :/
new_cols = {'PARID': 'ParcelID', 'TAX_DISTR': 'TaxDist','LIV_UNITS': 'LivUnits', 'OWNER1': 'Owner', 'STRUCT_YR': 'StructYr',
            'TOT_ASSESS': 'TotAssess','TOT_APPR': 'TotAppr', 'TAXYEAR': 'TaxYear','NBHD' : 'NbrHood', 'LUC':'LUCode',
           'LAND_APPR' : 'LandAppr', 'PROP_CLASS': 'PropClass', 'STRUCT_FLR': 'StructFlr', 'SITUS':'Address' }
Parcels_10.rename(columns=new_cols, inplace=True)
Parcels_09.rename(columns=new_cols, inplace=True)  

# a subset of columns to analyze.
simplified_cols = ['ParcelID', 'TaxDist', 'Owner', 'TotAppr', 'TotAssess', 'LivUnits']

In [163]:
Parcels_09.columns

Index(['Address', 'TAXPIN', 'ParcelID', 'ATRPIN', 'TaxDist', 'Owner', 'OWNER2',
       'ADD2', 'ADD3', 'ADD4', 'ADD5', 'LUCode', 'NbrHood', 'PropClass',
       'CLASS', 'TotAppr', 'TotAssess', 'IMPR_APPR', 'LandAppr', 'FUL_EX_COD',
       'StructFlr', 'StructYr', 'TIEBACK', 'TaxYear', 'STATUS_COD', 'LivUnits',
       'PCODE', 'UNIT_NUM', 'EXTVER', 'Shape__Area', 'Shape__Length'],
      dtype='object')

In [122]:
Parcels_11.columns

Index(['ParcelID', 'TaxYear', 'Address', 'AddrNumber', 'AddrPreDir',
       'AddrStreet', 'AddrSuffix', 'AddrPosDir', 'AddrUntTyp', 'AddrUnit',
       'Owner', 'OwnerAddr1', 'OwnerAddr2', 'TaxDist', 'TotAssess',
       'LandAssess', 'ImprAssess', 'TotAppr', 'LandAppr', 'ImprAppr', 'LUCode',
       'ClassCode', 'LivUnits', 'LandAcres', 'NbrHood', 'Subdiv', 'SubdivNum',
       'SubdivLot', 'SubdivBlck', 'Shape__Area', 'Shape__Length'],
      dtype='object')

In [13]:
# drop useless columns or duplicate data
dup_col = ['DIGEST', 'VAL_ACRES', 'TAXPIN', 'ATRPIN']
Parcels_09.drop(dup_col, axis='columns', inplace=True)
Parcels_10.drop(dup_col, axis='columns', inplace=True)
# drop rows with no ['TaxYear'] data
dfs = [Parcels_09, Parcels_10, Parcels_11, Parcels_12, Parcels_13, Parcels_14, Parcels_15
        , Parcels_16, Parcels_17, Parcels_18, Parcels_19, Parcels_20, Parcels_21, Parcels_22]
# Check columns of interest for NaN values, and drop that row
for df in dfs:
        df.dropna(subset=['TaxYear', 'ParcelID', 'TaxDist', 'TotAppr', 'TotAssess', 'LivUnits'], inplace=True)
        df.astype({'TaxYear': 'int16'})
# check that it worked
Parcels_09[pd.isna(Parcels_09['TaxYear'])==True]

"\nfor df in dfs:\n        df.dropna(subset=['TaxYear', 'ParcelID', 'TaxDist', 'TotAppr', 'TotAssess', 'LivUnits'], inplace=True)\n        df.astype({'TaxYear': 'int16'})\n# check that it worked\nParcels_09[pd.isna(Parcels_09['TaxYear'])==True]\n"

## [Fulton County Tax Districts](https://fultonassessor.org/wp-content/uploads/sites/16/2018/05/Fulton-County-Tax-Districts-2018.pdf)
05 City of Atlanta  
05A City of Atlanta – Downtown CID  
05B City of Atlanta – Buckhead CID  
05C City of Atlanta – Midtown CID  
05D City of Atlanta – Downtown CID/Midtown CID  
05E City of Atlanta – Downtown CID/Midtown CID/Eastside TAD  
05F City of Atlanta – Downtown CID/Eastside TAD  
05G City of Atlanta – Beltline TAD/Midtown CID  
05H City of Atlanta – Airport West CID  
05I City of Atlanta – Airport West CID/Princeton Lakes TAD  
05J City of Atlanta – Little Five Points CID  
05K City of Atlanta - Westside CID  
05L City of Atlanta – Westside CID/Beltline TAD  
05P City of Atlanta – Stadium TAD  
05Q City of Atlanta – Campbellton TAD  
05R City of Atlanta – Metropolitan TAD  
05S City of Atlanta – Hollowell TAD  
05T City of Atlanta – Beltline TAD  
05U City of Atlanta – Princeton Lakes TAD  
05V City of Atlanta – Perry Bolton TAD  
05W City of Atlanta – Eastside TAD  
05X City of Atlanta – Atlantic Station TAD  
05Y City of Atlanta – Downtown CID/Westside TAD  
05Z City of Atlanta – Westside TAD  
10 City of Alpharetta  
10A City of Alpharetta – North Fulton CID    
15 City of College Park  
15H City of College Park –Airport West CID  
15Z City of College Park – College Park TAD  
20 City of East Point  
20H City of East Point – Airport West CID  
20I City of East Point- East Point TAD #2/Airport West CID  
20J City of East Point – East Point TAD/Airport West CID  
20Y City of East Point – East Point TAD #2  
20Z City of East Point – East Point TAD  
25 City of Fairburn  
25C City of Fairburn – South Fulton CID  
30 City of Hapeville  
30A City of Hapeville – Airport West CID  
30B City of Hapeville – Hapeville TAD/Airport West CID  
30Z City of Hapeville – Hapeville TAD  
35 City of Mountain Park  
40 City of Palmetto  
40C City of Palmetto – South Fulton CID  
45 City of Roswell  
45A City of Roswell – North Fulton CID  
50 City of Union City  
50C City of Union City – South Fulton CID   
50Z City of Union City – Union City TAD  
55 City of South Fulton  
55C City of South Fulton – South Fulton CID  
55D City of South Fulton – Fulton Industrial Boulevard CID  
55H City of South Fulton – Airport West CID  
56 City of Milton  
56A City of Milton – North Fulton CID  
57 City of Johns Creek  
59 City of Sandy Springs  
59B City of Sandy Springs – Fulton Perimeter CID  
65 City of Chattahoochee Hills  
70 Fulton Industrial District  
70D Fulton Industrial District – Fulton Industrial Boulevard CID  
Revised June 9, 2016  

In [21]:
Parcels_11.count(numeric_only=True)

TaxYear          342983
AddrNumber       342921
TotAssess        342978
LandAssess       342970
ImprAssess       342970
TotAppr          342978
LandAppr         342978
ImprAppr         342970
LivUnits         336732
LandAcres        339442
Shape__Area      342981
Shape__Length    342981
dtype: int64

In [93]:
Parcels_11['Owner'].isna().sum()

96

In [25]:
len(Parcels_11.count(numeric_only=True))

12

## Cleaning Functions

In [53]:
# probably won't use this. 
def city_select(df_list, tax_ID_list):
    '''Selects city from annual data based on tax ID codes for that city.
        Drops rows that do not contain the tax ID. Overwrites original dataframe'''
    for df in df_list:
        df = df[df['TaxDist'].isin(tax_ID_list)]

### Cleaning the data
Our dats has 31 columns in it  and only 12 have numeric data. Since I plan on concatenating 13 years of data together I'd like to remove as much unnecesary data as possible. I have a list of simplified columns already, which pares it down to just the columns I'm immediately interested in but I'd like to leave my options open in case I uncover an interesting connection that may have links to other columns. I think the right move here is to remove columns that are mostly null values. In the code below we can see that 'AddrPreDir', 'AddrUntTyp', 'AddrUnit', 'Subdiv', 'SubdivNum', 'SubdivLot', and 'SubdivBlck' have drastically less data than the rest of the columns. 

## Working with all data in one dataframe

In [168]:
# All property data from 2009 - 2022
all_property_data = pd.concat(dfs)

In [169]:
# Hold on to your butts
all_property_data.to_csv('PropertyData2009-2022.csv', sep=',')

In [170]:
all_property_data.columns

Index(['Address', 'ParcelID', 'TaxDist', 'Owner', 'OWNER2', 'ADD2', 'ADD3',
       'ADD4', 'ADD5', 'LUCode', 'NbrHood', 'PropClass', 'CLASS', 'TotAppr',
       'TotAssess', 'IMPR_APPR', 'LandAppr', 'FUL_EX_COD', 'StructFlr',
       'StructYr', 'TIEBACK', 'TaxYear', 'STATUS_COD', 'LivUnits', 'PCODE',
       'UNIT_NUM', 'EXTVER', 'Shape__Area', 'Shape__Length', 'GID',
       'AddrNumber', 'AddrPreDir', 'AddrStreet', 'AddrSuffix', 'AddrPosDir',
       'AddrUntTyp', 'AddrUnit', 'OwnerAddr1', 'OwnerAddr2', 'LandAssess',
       'ImprAssess', 'ImprAppr', 'ClassCode', 'LandAcres', 'Subdiv',
       'SubdivNum', 'SubdivLot', 'SubdivBlck', 'FeatureID', 'ExCode'],
      dtype='object')

In [145]:
all_property_data.isna().sum()

SITUS            4149855
TAXPIN           4149855
ParcelID               0
ATRPIN           4149855
TaxDist                0
Owner                208
OWNER2           4149855
ADD2             4149855
ADD3             4149855
ADD4             4149855
ADD5             4149855
LUCode                 0
NbrHood                0
PropClass        4149855
CLASS            4149855
TotAppr                0
TotAssess              0
IMPR_APPR        4149855
LandAppr               0
FUL_EX_COD       4149855
StructFlr        4149855
StructYr         4149855
TIEBACK          4149855
TaxYear                0
STATUS_COD       4149855
LivUnits               0
PCODE            4149855
UNIT_NUM         4149855
EXTVER           4149855
Shape__Area            3
Shape__Length          3
GID              4493752
Address           688497
AddrNumber        689069
AddrPreDir       4835180
AddrStreet        688558
AddrSuffix        954623
AddrPosDir       3240690
AddrUntTyp       4461177
AddrUnit         4262183


In [151]:
# DON'T RUN 4.8M rows, 53 columns, 2GB dataframe bricks 16 GB of ram for 3-4 min. Oof.wav
# not really useful anyway
# all_property_data[~all_property_data.isna()]

,SITUS,TAXPIN,ParcelID,ATRPIN,TaxDist,Owner,OWNER2,ADD2,ADD3,ADD4,...,ImprAssess,ImprAppr,ClassCode,LandAcres,Subdiv,SubdivNum,SubdivLot,SubdivBlck,FeatureID,ExCode
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,665 FALLS LAKE DR,11 01910100086,11 019101000864,11 -0191-0100-086-4,57,JOEL KENNETH H &,,665 FALLS LAKE DR,ALPHARETTA GA 30022,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,809 PISTACE CT,11 00700014087,11 007000140873,11 -0070-0014-087-3,57,MADISON PARK LLC,,1805 OLD ALABAMA RD SUITE 230,ROSWELL GA 30076,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021 WESTBOURNE WAY,11 02000071116,11 020000711163,11 -0200-0071-116-3,57,BALL HOWARD,,2021 WESTBOURNE WAY,ALPHARETTA GA 30022,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10083 TRUMPET PARK,11 00700033109,11 007000331092,11 -0070-0033-109-2,57,MADISON PARK LLC,,1805 OLD ALABAMA RD SUITE 230,ROSWELL GA 30076,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10914 ESTATES CIR,11 01600080191,11 016000801913,11 -0160-0080-191-3,10,TB SIX LLC,,1210 NORTHCLIFF TRC,ROSWELL GA 30072,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7409417,NaN,NaN,17 010800011812,NaN,05X,STEVERSON SUMMER,NaN,NaN,NaN,NaN,...,85680.0,214200.0,R3,0.0191,NaN,R,NaN,NaN,17 010800011812,NaN
7409418,NaN,NaN,17 010800011820,NaN,05X,SHEHU SHEHU SULE,NaN,NaN,NaN,NaN,...,76720.0,191800.0,R3,0.0158,NaN,R,NaN,NaN,17 010800011820,NaN
7409419,NaN,NaN,17 010800011838,NaN,05X,PERRIE DAVID,NaN,NaN,NaN,NaN,...,78800.0,197000.0,R3,0.0166,NaN,R,NaN,NaN,17 010800011838,NaN


In [149]:
all_property_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4838352 entries, 1 to 7409421
Data columns (total 53 columns):
 #   Column         Dtype  
---  ------         -----  
 0   SITUS          object 
 1   TAXPIN         object 
 2   ParcelID       object 
 3   ATRPIN         object 
 4   TaxDist        object 
 5   Owner          object 
 6   OWNER2         object 
 7   ADD2           object 
 8   ADD3           object 
 9   ADD4           object 
 10  ADD5           object 
 11  LUCode         object 
 12  NbrHood        object 
 13  PropClass      object 
 14  CLASS          object 
 15  TotAppr        float64
 16  TotAssess      float64
 17  IMPR_APPR      float64
 18  LandAppr       float64
 19  FUL_EX_COD     object 
 20  StructFlr      float64
 21  StructYr       float64
 22  TIEBACK        object 
 23  TaxYear        float64
 24  STATUS_COD     object 
 25  LivUnits       float64
 26  PCODE          object 
 27  UNIT_NUM       float64
 28  EXTVER         float64
 29  Shape__Area   

In [171]:
# pulls out just columns with useful amounts of or potentially interesting data
useful_df = all_property_data[['TaxYear', 'Owner', 'ParcelID', 'Address', 'AddrStreet','TaxDist', 'LUCode', 'NbrHood',
                                'TotAppr', 'TotAssess', 'LandAppr', 'LandAssess', 'LivUnits', 'LandAcres']] 

In [204]:
# fixes data types to pare down memory usage
useful_df = useful_df.astype({'TaxYear' : 'int16', 'TotAppr' : 'int32', 'TotAssess' : 'int32', 'LandAppr' : 'int32'}) 

In [205]:
useful_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4838352 entries, 1 to 7409421
Data columns (total 14 columns):
 #   Column      Dtype  
---  ------      -----  
 0   TaxYear     int16  
 1   Owner       object 
 2   ParcelID    object 
 3   Address     object 
 4   AddrStreet  object 
 5   TaxDist     object 
 6   LUCode      object 
 7   NbrHood     object 
 8   TotAppr     int32  
 9   TotAssess   int32  
 10  LandAppr    int32  
 11  LandAssess  float64
 12  LivUnits    float64
 13  LandAcres   float64
dtypes: float64(3), int16(1), int32(3), object(7)
memory usage: 502.9+ MB


In [207]:
# Selects just data that has Atlanta tax IDs
atlanta_df = useful_df[useful_df['TaxDist'].isin(atlanta_tax_IDs)]
atlanta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068625 entries, 10 to 7409421
Data columns (total 14 columns):
 #   Column      Dtype  
---  ------      -----  
 0   TaxYear     int16  
 1   Owner       object 
 2   ParcelID    object 
 3   Address     object 
 4   AddrStreet  object 
 5   TaxDist     object 
 6   LUCode      object 
 7   NbrHood     object 
 8   TotAppr     int32  
 9   TotAssess   int32  
 10  LandAppr    int32  
 11  LandAssess  float64
 12  LivUnits    float64
 13  LandAcres   float64
dtypes: float64(3), int16(1), int32(3), object(7)
memory usage: 201.2+ MB


In [208]:
atlanta_df.to_csv('CleanedAtlantaPropertyData_2009-2022.csv', sep=',')

## Exploring the Data  
This section is a bit haphazard. These cells were just things I ran to explore the data. Once I got a feel for how things were organized and found the years in which their style was changed I was able to actually clean the data. Which is reflected in the cells above.

In [23]:
# Parcels_12.columns

In [24]:
# Parcels_13.columns

In [25]:
# Parcels_14.columns

In [26]:
# Parcels_15.columns

In [29]:
# Parcels_10.columns

In [201]:
useful_df[useful_df['TotAppr'] == useful_df['TotAppr'].max()]
# 1.3 billion dollar cemetary. Lol, wut?

,TaxYear,Owner,ParcelID,Address,AddrStreet,TaxDist,LUCode,NbrHood,TotAppr,TotAssess,LandAppr,LandAssess,LivUnits,LandAcres
OBJECTID,,,,,,,,,,,,,,
296406,2013,ANTIOCH BAPTIST CHURCH OF ATL,14 004100080632,0 MC DONOUGH BLVD SE,MC DONOUGH,05T,601,14281,1.261313e+09,0.0,1.261313e+09,504525280.0,0.0,1.13999
320506,2013,ANTIOCH BAPTIST CHURCH OF ATL,14 004100080632,0 MC DONOUGH BLVD SE,MC DONOUGH,05T,601,14281,1.261313e+09,0.0,1.261313e+09,504525280.0,0.0,1.13999
103810,2014,ANTIOCH BAPTIST CHURCH OF ATL,14 004100080632,0 MC DONOUGH BLVD SE,MC DONOUGH,05T,601,14281,1.261313e+09,0.0,1.261313e+09,504525280.0,0.0,1.13999
317882,2014,ANTIOCH BAPTIST CHURCH OF ATL,14 004100080632,0 MC DONOUGH BLVD SE,MC DONOUGH,05T,601,14281,1.261313e+09,0.0,1.261313e+09,504525280.0,0.0,1.13999


In [35]:
Parcels_11.isna().sum()

ParcelID              0
TaxYear               0
Address               0
AddrNumber           62
AddrPreDir       342775
AddrStreet            3
AddrSuffix        22171
AddrPosDir       207883
AddrUntTyp       317827
AddrUnit         295379
Owner                96
OwnerAddr1          123
OwnerAddr2         1568
TaxDist               0
TotAssess             5
LandAssess           13
ImprAssess           13
TotAppr               5
LandAppr              5
ImprAppr             13
LUCode                5
ClassCode             0
LivUnits           6251
LandAcres          3541
NbrHood               0
Subdiv           341068
SubdivNum        292703
SubdivLot        318682
SubdivBlck       338803
Shape__Area           2
Shape__Length         2
dtype: int64

In [23]:
# find columns with no data or 'na'. Creates pd.series with number of NaN values per df['column']
parcel_na = Parcels_11.isna().sum()
parcel_na[parcel_na > 50_000]

AddrPreDir    342775
AddrPosDir    207883
AddrUntTyp    317827
AddrUnit      295379
Subdiv        341068
SubdivNum     292703
SubdivLot     318682
SubdivBlck    338803
dtype: int64

In [73]:
parcel_na[parcel_na > 50_000].keys()

Index(['AddrPreDir', 'AddrPosDir', 'AddrUntTyp', 'AddrUnit', 'Subdiv',
       'SubdivNum', 'SubdivLot', 'SubdivBlck'],
      dtype='object')

In [71]:
parcel_na

ParcelID              0
TaxYear               0
Address               0
AddrNumber           62
AddrPreDir       342775
AddrStreet            3
AddrSuffix        22171
AddrPosDir       207883
AddrUntTyp       317827
AddrUnit         295379
Owner                96
OwnerAddr1          123
OwnerAddr2         1568
TaxDist               0
TotAssess             5
LandAssess           13
ImprAssess           13
TotAppr               5
LandAppr              5
ImprAppr             13
LUCode                5
ClassCode             0
LivUnits           6251
LandAcres          3541
NbrHood               0
Subdiv           341068
SubdivNum        292703
SubdivLot        318682
SubdivBlck       338803
Shape__Area           2
Shape__Length         2
dtype: int64

In [47]:
# searches all columns and returns those with more than 50,000 NaN values
Parcels_11.loc[:, Parcels_11.isna().sum() > 50_000].columns

Index(['AddrPreDir', 'AddrPosDir', 'AddrUntTyp', 'AddrUnit', 'Subdiv',
       'SubdivNum', 'SubdivLot', 'SubdivBlck'],
      dtype='object')

In [53]:
Parcels_11.loc[:,Parcels_11.isna().sum() > 10_000].keys()

Index(['AddrPreDir', 'AddrSuffix', 'AddrPosDir', 'AddrUntTyp', 'AddrUnit',
       'Subdiv', 'SubdivNum', 'SubdivLot', 'SubdivBlck'],
      dtype='object')

In [67]:
# 25k seems to be useful here. Below this and I lose AddrSuffix, which I would like to keep despite the NaNs
Parcels_11.loc[:, Parcels_11.isna().sum() > 25_000]

,AddrPreDir,AddrPosDir,AddrUntTyp,AddrUnit,Subdiv,SubdivNum,SubdivLot,SubdivBlck
OBJECTID,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,24400,1,NaN
5,NaN,NaN,NaN,NaN,NaN,24400,1,NaN
...,...,...,...,...,...,...,...,...
343085,NaN,NaN,NaN,NaN,NaN,24209,130,NaN
343086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# are these Apts or is this nonsense?
Parcels_09.LivUnits[Parcels_09['LivUnits'] > 100]

OBJECTID
1650      999
2562      242
7471      269
11034     280
11131     101
         ... 
343087    200
343088    200
343089    200
343090    200
343091    200
Name: LivUnits, Length: 603, dtype: int64

In [50]:
# view of parcels that have an Atlanta Tax ID
''' Still not sure if I want to keep these separate or concat before I analyze. I think I want to concat but then I have to
    do more logic to fine annual data. Here it's already split. I suppose I could just create views onto each year from the 
    main df... hmm.
'''
# replaced cell with city_select function
# replaced again by using on large dataframe
atlanta_09DF = Parcels_09[Parcels_09['TaxDist'].isin(atlanta_tax_IDs)]
'''
atlanta_10DF = Parcels_10[Parcels_10['TaxDist'].isin(atlanta_tax_IDs)]
atlanta_11DF = Parcels_11[Parcels_11['TaxDist'].isin(atlanta_tax_IDs)]
atlanta_12DF = Parcels_12[Parcels_12['TaxDist'].isin(atlanta_tax_IDs)]
atlanta_13DF = Parcels_13[Parcels_13['TaxDist'].isin(atlanta_tax_IDs)]
atlanta_14DF = Parcels_14[Parcels_14['TaxDist'].isin(atlanta_tax_IDs)]
atlanta_15DF = Parcels_15[Parcels_15['TaxDist'].isin(atlanta_tax_IDs)]
'''

"\natlanta_10DF = Parcels_10[Parcels_10['TaxDist'].isin(atlanta_tax_IDs) == True]\natlanta_11DF = Parcels_11[Parcels_11['TaxDist'].isin(atlanta_tax_IDs) == True]\natlanta_12DF = Parcels_12[Parcels_12['TaxDist'].isin(atlanta_tax_IDs) == True]\natlanta_13DF = Parcels_13[Parcels_13['TaxDist'].isin(atlanta_tax_IDs) == True]\natlanta_14DF = Parcels_14[Parcels_14['TaxDist'].isin(atlanta_tax_IDs) == True]\natlanta_15DF = Parcels_15[Parcels_15['TaxDist'].isin(atlanta_tax_IDs) == True]\n"